In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
%matplotlib inline

from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 입력데이터

In [3]:
df2 = pd.read_csv('final_data_19612.csv', sep='\t', encoding='UTF-8')
df = df2.loc[df2['대상 선정']=='O',:]

In [4]:
df = df.fillna('N')

userdic = pd.read_csv('userdic_1906122.csv', sep='\t', encoding='UTF-8')
userdic['User Dic.'][0:1]
userdic = userdic.rename(columns = {'User Dic.':'standard'})
userdic = userdic.rename(columns = {'동의어':'missprint'})
userdic = userdic.rename(columns = {'불용단어':'stops'})
userdic_list = userdic.rename(columns = {'User Dic.':'standard'})
userdic_list = userdic.rename(columns = {'동의어':'missprint'})
userdic_list = userdic.rename(columns = {'불용단어':'stops'})
userdic_list = userdic_list.fillna('')
userdic = userdic.fillna('')
replace_word = userdic.iloc[:,1:3]
replace_word = replace_word[replace_word.missprint != '']

In [5]:
replaceword_list =  replace_word['standard'].tolist()
missword_list = replace_word['missprint'].tolist()
print(len(replaceword_list))
print(len(missword_list))

757
757


In [6]:
for i in range(len(replaceword_list)):
    df['clm_prtl_sbc'] = df['clm_prtl_sbc'].replace(missword_list[i],replaceword_list[i],regex=True)
#regex=True 설정을 주면 조사가 붙어있던 말던 단어를 다 바꿔준다.

In [16]:
df.shape

(4331, 16)

In [7]:
df[df['clm_prtl_sbc'].isnull()]

Empty DataFrame
Columns: [std_clm_type_cd, ro_y, clm_jgmt_dt, ceq_caus_cd, ceq_psit_cd, ceq_psit_nm, trim_plnt_cd, prj_vehl_cd, mdl_cd, part_sys_cd, repn_part_cd, clm_prtl_sbc, 대상 선정, 구분, 책임구, 원인]
Index: []

## 텍스트 전처리

In [8]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
# df['result_pred'] = df['검토결과'].astype(str).copy()
# df['target_pred'] = df['대상 선정'].astype(str).copy()
# df['charge_pred'] = df['책임구'].astype(str).copy()
# df['division_pred'] = df['구분'].astype(str).copy()

### (1) 오타, 동의어 처리(replace)

### (2) 사용자 단어 정의(User dictionary)

for word in replaceword_list: 
    twitter.add_dictionary(word, 'Noun') # for문을 통해 dictionary에 단어들을 Noun으로 추가.

# twitter.pos(df['clm_prtl_sbc'][1]) # '리어', '미장착'이 Noun으로 잘 정의되었는지 확인.

### (3) 명사 추출(Twitter)

def nouns(text):
    text = str(text)
    text = twitter.nouns(text)
    return text

df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nouns)

### (4) 특수문자, 띄어쓰기 전처리

def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) 
    text = re.sub(' +', ' ', text)
    return text

# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['(LH)','(RH)','(사진첨부)','(우)','(좌)','[C]','A/S','C','C-10000','C-10000','F/LH','F/RH','FR','FRT',
             'Frt','frt','L/H','L/H리어','LH','LH/RH','LH쪽','R/H','R/H리어','R/LH','R/RH','RH','RH쪽','Rr','RR','rr',
             '결과','고객','고객에게','공임','광주','그랜저','그랜져','낫','낮','넥쏘','니로','당','당차량','당차량은','대리점',
             '동영상참조','레이','렌트','리무진','리어LH','리어RH','모닝','모하비','밤','벨로스터','사진','사진참조','사진첨부',
             '상기','상기차량은','새벽','새차','서비스','소하','스타렉스','스팅어','스포티지','신차','신차','심야','싼타페',
             '써비스','쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','아침','야간','언덕','엑센트','열','울산','원인',
             '임시 넘버','임시 번호','입고','자동차','저머검내용','점검/원인','점검결과','점검내용','점검및원인','점검및조치내용',
             '점검점검','점검점검내용','조치결과','조치내용','주간','주재원','주제원','주차장','증상','지하','진입','차량',
             '차량은','철도','첨부','첨부파일','청구','출고','카 마스터','카니발','카렌스','코나','클레임','투싼',
             '팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상','현 상','현산','현상','현상및점검',
             '현상및조치','화성'
]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

df['clm_prtl_sbc'].head()

Wall time: 290 ms
Wall time: 320 ms


12     인도 후 오디오 작동 시 주파수 및 이음 발 생으로 불량 제기 점검 내용 및 점검...
14     주행중 엔진 경고등 점등 점검 내용 및 스캔 점검 효율 저하 발생 임시 코드 주행...
15     테일게이트 게이트 트림 뜨르르르 이상 이음 발 생으로 테일게이트 게이트 트림 탈착...
19                   조수석 사이드 어백 경고등 점등 커넥터 터 체결 불량 재체결 
24                               작동 불량 커넥터 터 조립 불량 재조립 
Name: clm_prtl_sbc, dtype: object

In [9]:
df.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
12               W  2017     20171010         ZZ7         B11  조립 문제_미장착/오장착   
14               W  2018     20180725         ZZ7         B11  조립 문제_미장착/오장착   
15               W  2015     20151114         ZZ3         B11  조립 문제_미장착/오장착   

   trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
12         KV11          CK     J5          AC        91500   
14         KV11          YP     A9          EE        28291   
15         KV11          YP     A9          TT        81750   

                                         clm_prtl_sbc 대상 선정       구분   책임구  \
12   인도 후 오디오 작동 시 주파수 및 이음 발 생으로 불량 제기 점검 내용 및 점검...     O  기능(커넥터)  생산공장   
14   주행중 엔진 경고등 점등 점검 내용 및 스캔 점검 효율 저하 발생 임시 코드 주행...     O  기능(커넥터)  생산공장   
15   테일게이트 게이트 트림 뜨르르르 이상 이음 발 생으로 테일게이트 게이트 트림 탈착...     O   실내부품이음  생산공장   

     원인  
12  미결선  
14   기타  
15   이물

## X, y 정의

In [10]:
x = df['clm_prtl_sbc']
y = df['구분']

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np


max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)                   # X의 각 행에 토큰화를 수행
sequences = tokenizer.texts_to_sequences(x) # 단어를 숫자인덱스로 변환하여 저장

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았어요.' % len(word_index))

4613개의 고유한 토큰을 찾았어요.


In [12]:
sequences[:10]

[[505,
  7,
  374,
  12,
  1,
  1494,
  9,
  3,
  29,
  25,
  6,
  553,
  2,
  18,
  9,
  2,
  1494,
  746,
  1739,
  1494,
  238,
  11,
  253,
  1495,
  1740,
  574,
  388,
  79,
  7,
  374,
  9,
  381,
  11,
  16,
  147,
  185,
  7,
  1496,
  589,
  11,
  23,
  46,
  6,
  170,
  10,
  18,
  1496,
  589,
  2706,
  46,
  7,
  1739,
  11,
  16,
  12],
 [8,
  40,
  15,
  14,
  2,
  18,
  9,
  73,
  2,
  506,
  375,
  4,
  664,
  39,
  30,
  2,
  15,
  14,
  10,
  18,
  9,
  528,
  17,
  5,
  7,
  1,
  8,
  16,
  665,
  1741,
  97,
  542,
  145,
  6],
 [65,
  67,
  36,
  2707,
  22,
  3,
  29,
  25,
  65,
  67,
  36,
  175,
  11,
  1,
  543,
  175,
  590,
  22,
  3,
  4,
  11,
  13,
  515,
  270,
  65,
  67,
  36,
  53,
  422,
  36,
  423,
  53,
  189,
  168,
  175,
  47,
  7,
  213],
 [32, 94, 1332, 15, 14, 24, 61, 46, 6, 129],
 [12, 6, 24, 61, 53, 6, 213],
 [424, 145, 8, 28, 57, 117, 145, 6],
 [21,
  20,
  12,
  6,
  2,
  1,
  95,
  8,
  97,
  12,
  6,
  105,
  11,
  989,
  96,
  36,
  

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

x = sequences
print(max(len(l) for l in x))
plt.hist([len(s) for s in x], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

168


In [14]:
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(y)                     # y의 각 행에 토큰화를 수행
sequences2 = tokenizer2.texts_to_sequences(y)  # 단어를 숫자인덱스로 변환하여 저장

word_index2 = tokenizer2.word_index
print('%s개의 고유한 토큰을 찾았어요.' % len(word_index2))

4개의 고유한 토큰을 찾았어요.


In [15]:
y = sequences2
print(max(len(l) for l in y))
plt.hist([len(s) for s in y], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

2


In [17]:
maxlen = 168
training_samples = 3898
test_samples = 10000

In [18]:
from keras.preprocessing import sequence
data_x = pad_sequences(sequences, maxlen=maxlen)
data_y = pad_sequences(sequences2)

In [19]:
data_x

array([[   0,    0,    0, ...,   11,   16,   12],
       [   0,    0,    0, ...,  542,  145,    6],
       [   0,    0,    0, ...,   47,    7,  213],
       ...,
       [   0,    0,    0, ...,  463,   37,   13],
       [   0,    0,    0, ...,  319,    7,   13],
       [   0,    0,    0, ..., 2705,  180, 4613]])

In [20]:
data_y

array([[1, 2],
       [1, 2],
       [0, 3],
       ...,
       [0, 3],
       [0, 3],
       [0, 3]])

In [21]:
# labels = np.asarray(y)
print('x 데이터 텐서 크기:', data_x.shape)
print('y 데이터 텐서 크기:', data_y.shape)

x 데이터 텐서 크기: (4331, 168)
y 데이터 텐서 크기: (4331, 2)


In [22]:
indices = np.arange(data_x.shape[0])
indices2 = np.arange(data_y.shape[0])
np.random.shuffle(indices)
np.random.shuffle(indices2)
data_x = data_x[indices]
data_y = data_y[indices2]

In [23]:
x_train = data_x[:training_samples]
y_train = data_y[:training_samples]
x_test = data_x[training_samples: training_samples + test_samples]
y_test = data_y[training_samples: training_samples + test_samples]

In [24]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3898, 168)
(3898, 2)
(433, 168)
(433, 2)


In [25]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

[[  0   0   0 ...   5  10  13]
 [  0   0   0 ...   9  99   5]
 [  0   0   0 ...  64   5  47]
 ...
 [  0   0   0 ...  21 110   5]
 [  0   0   0 ...  29  25   5]
 [  0   0   0 ...  16  12  11]]
[[1 2]
 [0 3]
 [0 4]
 ...
 [0 3]
 [0 3]
 [1 2]]
[[  0   0   0 ...   7  16  12]
 [  0   0   0 ...  18   5 276]
 [  0   0   0 ...   6   3   4]
 ...
 [  0   0   0 ... 114  47 275]
 [  0   0   0 ...   7  16  12]
 [  0   0   0 ... 114   7  16]]
[[1 2]
 [1 2]
 [0 4]
 [1 2]
 [1 2]
 [0 3]
 [0 3]
 [0 3]
 [1 2]
 [1 2]
 [0 3]
 [0 3]
 [0 3]
 [0 4]
 [0 3]
 [1 2]
 [1 2]
 [1 2]
 [0 3]
 [0 4]
 [0 4]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [0 4]
 [0 4]
 [1 2]
 [0 3]
 [0 4]
 [1 2]
 [0 4]
 [1 2]
 [0 3]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [0 3]
 [0 4]
 [1 2]
 [0 3]
 [0 3]
 [0 3]
 [1 2]
 [0 3]
 [1 2]
 [1 2]
 [1 2]
 [0 3]
 [0 3]
 [0 3]
 [1 2]
 [1 2]
 [1 2]
 [0 4]
 [0 3]
 [0 3]
 [0 3]
 [1 2]
 [0 3]
 [0 3]
 [1 2]
 [0 3]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [0 3]
 [1 2]
 [1 2]
 [0 3]
 [0 3]
 [1 2]
 [1 2]
 [0 3]
 [1 2]
 [1 2]
 

In [26]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,Flatten,SimpleRNN

max_features = 10000
# maxlen = 163
# embedding_dim = 163

model = Sequential()
model.add(Embedding(max_features,32))
model.add(SimpleRNN(32))
model.add(Dense(2,activation='sigmoid'))

# model.add(Embedding(max_features,embedding_dim,input_length=maxlen))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 322,146
Trainable params: 322,146
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train,y_train,epochs=10,batch_size=128,validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 3118 samples, validate on 780 samples
Epoch 1/10
3118/3118 [==============================] - 27s 9ms/step - loss: 0.0801 - acc: 0.2404 - val_loss: -0.9536 - val_acc: 0.2551
Epoch 2/10
3118/3118 [==============================] - 54s 17ms/step - loss: -2.0053 - acc: 0.2452 - val_loss: -3.2693 - val_acc: 0.2442
Epoch 3/10
3118/3118 [==============================] - 5s 2ms/step - loss: -4.0455 - acc: 0.2611 - val_loss: -5.0414 - val_acc: 0.2622
Epoch 4/10
3118/3118 [==============================] - 3s 827us/step - loss: -5.5137 - acc: 0.2579 - val_loss: -6.3161 - val_acc: 0.2596
Epoch 5/10
3118/3118 [==============================] - 3s 907us/step - loss: -6.6239 - acc: 0.2603 - val_loss: -7.3057 - val_acc: 0.2449
Epoch 6/10
3118/3118 [==============================] - 3s 830us/step - loss: -7.4812 - acc: 0.2611 - val_loss: -8.0841 - val_acc: 0.2468
Epoch 7/10
3118/3118 [==============================] - 2s 792us/step - loss: -8.

In [28]:
import matplotlib.pyplot as plt

In [29]:
print(history.history['acc'])
print(history.history['val_acc'])
print(history.history['loss'])
print(history.history['val_loss'])

[0.24037844768466668, 0.2451892237467518, 0.2610647850422006, 0.2578576009880663, 0.2602629891338067, 0.26106478501352615, 0.25192431035028046, 0.26090442587581486, 0.26090442579934964, 0.2609044257228844]
[0.2551282049753727, 0.24423076915435302, 0.2621794871794872, 0.25961538476821705, 0.24487179479537866, 0.24679487171845557, 0.2551282049753727, 0.24487179479537866, 0.24487179479537866, 0.24487179479537866]
[0.08006288333914845, -2.0052950984778657, -4.045539670911181, -5.513731851571642, -6.623922585370527, -7.481215954132765, -8.205228783520006, -8.902486036495187, -9.578364443213149, -10.251366141822722]
[-0.9536132806386703, -3.269331301175631, -5.041440252157358, -6.31611128831521, -7.305686363807092, -8.084066669757549, -8.808751008449457, -9.507750809498322, -10.196045371813652, -10.869184875488282]


In [30]:
print(round(np.mean(history.history['acc'])*100,2))
print(round(np.mean(history.history['val_acc'])*100,2))

25.6
25.03


In [31]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [32]:
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [33]:
# model.load_weights('pre_trained_model.h5')
model.evaluate(x_test, y_test)

433/433 [==============================] - 0s 182us/step


[-10.467320199926785, 0.2713625866394946]